# Quadratura Gaussiana

In [214]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

# Quadratura Gaussiana com Ajuda

In [215]:
def quadgausseasy(start: float, end: float, func: Callable, n: int) -> float:
    """
        Calcula o integral de `func` de `start` a `end` usando quadratura gaussiana com `n` pontos.

        ### Retorno
        Devolve o valor do integral.
    """

    # Determinar zeros e pesos
    xx, ww = np.polynomial.legendre.leggauss(n)

    # Calcular valores úteis
    dif = end - start
    som = end + start

    # Fazer com que o integral seja de -1 a 1
    def f(x: (float | np.ndarray)) -> (float | np.ndarray):
        return func((x * dif + som) / 2)
    
    return  float((dif / 2) * np.sum(f(xx) * ww))

# Quadratura Gaussiana sem Ajuda

Vamos agora fazer quadratura gaussiana sem ajuda do `numpy`!

Vamos determinar os zeros e os pesos de várias formas!

In [216]:
def legendre(x: (float | np.ndarray), n: int) -> (float | np.ndarray):
    """
        Usa a relação de recorrência dos polinómios de Legendre para avaliar o polinómio de ordem `n` em `x`.
    """
    if n == 0:
        return 1
    
    elif n == 1:
        return x
    
    else:
        n -= 1
        return (2 * n + 1) / (n + 1) * x * legendre(x, n) - (n / (n + 1)) * legendre(x, n-1)


def central(func: Callable, h: float = 1e-4) -> Callable:
    """
        Calcula a derivada de `func` usando a diferença central com um passo de `h`.

        ### Retorno
        Devolve uma função `fprime` que recebe como argumento `x` e devolve a derivada de `func` em `x`.
    """

    return lambda x: (func(x + h/2) - func(x - h/2)) / h


def bissect(func: Callable, a: float = 0, b: float = 1, eps: float = 1e-6) -> float:
    """
        Resolve a equação `func(x) = 0` usando o método da bisseção no intervalo [a, b] parando quando o erro for menor que `eps`.

        ### Retorno
        O valor x que é solução da equação.
    """

    erro = eps + 1
    while abs(erro) > eps:
        c = (a+b)/2

        fa, fb, fc = func(a), func(b), func(c)

        # Verificar se algum dos valores é um zero
        if fa == 0:
            return a
        elif fb == 0:
            return b
        elif fc == 0:
            return c

        # Troca de sinal em [a, c]
        elif fa * fc < 0:
            b = c
        
        # Troca de sinal em [c, b]
        elif fc * fb < 0:
            a = c
        
        # Não há troca de sinal
        else:
            print(a, func(a))
            print(b, func(b))
            raise ValueError("O sinal de f é o mesmo em a e b.")

        erro = (b - a)/2
    
    return (a+b)/2


def secante(func: Callable, x0: np.ndarray, x1: np.ndarray, eps: float = 1e-6, maxI: int = 1000) -> (float | np.ndarray):
    """
        Resolve a equação `func(x) = 0` usando o método da secante começando com as estimativas `x0` e `x1` e parando quando o erro for menor que `eps` fazendo no máximo maxI iterações.

        ### Retorno
        O valor x que é solução da equação.
    """

    # Número de iterações e erro
    I = 0
    erro = eps + 1
    while abs(erro) > abs(eps):
        # Avaliar a função nos pontos em jogo
        fx0 = func(x0)
        fx1 = func(x1)

        # Evitar divisão por 0
        dif = np.where(x1 - x0 > 0, x1 - x0, 1e-30)

        # Determinar a derivada e o passo a dar
        derivada = (fx1 - fx0) / dif
        passo = fx1 / derivada

        # Calcular o erro
        erro = np.max(abs(passo))

        # Próximo passo
        x0, x1 = np.copy(x1), x1 - passo
        I += 1

        if I > maxI:
            raise ValueError(f"Não foi possível determinar os zeros com a precisão desejada com menos de maxI = {maxI} iterações!")
    
    return x1



def gausselim(Ao: np.ndarray, bbo: np.ndarray) -> np.ndarray:
    """
        Resolve o sistema de N equações definido por Ao * xx = bbo usando eliminação gaussiana com pivotagem parcial.

        ### Argumentos
        Ao: Matriz N por N
        bbo: Vetor com N entradas
        
        ### Retorno
        xx: Vetor com N entradas
    """

    # Evitar side effects
    A = np.copy(Ao)
    bb = np.copy(bbo)

    N = bb.size

    # Fazer pivotagem usando um mapa de endereços
    mm = np.arange(0, N)
    for line in range(N):
        # Posição do maior elemento da coluna (tendo em conta as trocas já feitas)
        coluna = A[mm[line:], line]
        index = list(abs(coluna)).index(max(abs(coluna))) + line

        # Trocar os elementos
        mm[line], mm[index] = mm[index], mm[line]

    # Eliminação Gaussiana
    for i in range(N-1):
        piv = A[mm[i]][i]
        for j in range(i+1, N):
            coef = A[mm[j]][i] / piv
            A[mm[j]] -= A[mm[i]] * coef
            bb[mm[j]] -= bb[mm[i]] * coef

    # Substituição Regressiva
    xx = np.zeros(N)
    xx[-1] = bb[mm[-1]] / A[mm[N-1]][N-1]
    for i in range(N-1, -1, -1):
        xx[i] = (bb[mm[i]] - (A[mm[i]][i+1:] @ xx[i+1:])) / A[mm[i]][i]
    
    return xx


def zeros(n: int, metodo: str, eps: float = 1e-6) -> np.ndarray:
    """
        Calcula os zeros do polinómio de Legendre de ordem `n` usando o método especificado por `metodo` com um erro máximo de `eps`.

        ### Retorno
        xx: Vetor com `n` entradas
    """

    # Estimativa inicial dos zeros (muito boa)
    kk = np.arange(1, n+1, 1)
    xx = (1 - 1/(8 * n**2) + 1/(8 * n**3)) * np.cos(np.pi / (4 * n + 2) * (4 * kk - 1))

    # Identificar os zeros usando o método da bisseção
    if metodo == "bissect":
        delta = 1 / (5 * n)
        for i in range(n):
            xx[i] = bissect(lambda x: legendre(x, n), xx[i] - delta, xx[i] + delta, eps)
    
    # Identificar os zeros usando o método da secante
    elif metodo == "secante":
        xx = secante(lambda x: legendre(x, n), np.ones_like(xx), xx, eps)

    return np.array(xx)


def pesos(n: int, xx: np.ndarray, metodo: str, h: float = 1e-6) -> np.ndarray:
    """
        Calcula os pesos da quadratura de Gauss-Legendre de ordem `n` sendo os valores dos zeros `xx` usando o método especificado por `metodo`.

        ### Retorno
        ww: Vetor com `n` entradas
    """ 

    ww = 0

    # Identificar os pesos usando eliminação gaussiana
    if metodo == "gausselim":
        # Criar a matriz de coeficientes
        A = np.zeros((n, n))
        for i in range(n):
            A[i] = legendre(xx, i)
        
        # Criar o vetor do lado direito da equação
        bb = np.zeros(n)
        bb[0] = 2

        # Resolver a equação e obter os pesos
        ww = gausselim(A, bb)
    
    # Identificar os pesos usando a derivada do polinómio de Legendre
    elif metodo == "derivada":
        ww = 2 / ((1 - xx ** 2) * (central(lambda x: legendre(x, n), h)(xx))**2)

    return np.array(ww)


def quadgauss1(start: float, end: float, func: Callable, n: int) -> float:
    """
        Calcula o integral de `func` de `start` a `end` usando quadratura gaussiana com `n` pontos. Calcula os zeros usando o método da bisseção e calcula os pesos resolvendo um sistema de equações lineares.

        ### Retorno
        Devolve o valor do integral.
    """

    # Determinar os n zeros do polinómio de Legendre de ordem n
    xx = zeros(n, "bissect")


    # Determinar os pesos
    ww = pesos(n, xx, "gausselim")


    # Fazer com que o integral seja de -1 a 1
    def f(x: (float | np.ndarray)) -> (float | np.ndarray):
        return func((x + 1) * (end - start) / 2 + start)
    
    return  float(((end - start) / 2) * np.sum(f(xx) * ww))


def quadgauss2(start: float, end: float, func: Callable, n: int, eps: float=1e-14, maxI: int=1000, h: float=1e-6) -> float:
    """
        Calcula o integral de `func` de `start` a `end` usando quadratura gaussiana com `n` pontos. Determina os zeros usando o método da secante com erro máximo igual a `eps` e número máximo de passos `I` e calcula os pesos usando a derivada dos polinómios de Legendre obtida pela diferença central com passo `h`.

        ### Retorno
        Devolve o valor do integral.
    """

    # Determinar os n zeros do polinómio de Legendre de ordem n
    xx = zeros(n, "secante")


    # Determinar os pesos
    ww = pesos(n, xx, "derivada")


    # Calcular valores úteis
    dif = end - start
    som = end + start


    # Fazer com que o integral seja de -1 a 1
    def f(x: (float | np.ndarray)) -> (float | np.ndarray):
        return func((x * dif + som) / 2)
    
    return  float((dif / 2) * np.sum(f(xx) * ww))

# Quadratura Gaussiana 2D

# Testar

Vamos fazer uma bateria de testes a este métodos!

In [217]:
from testes import testeintegral

In [218]:
print("Quadratura Gaussiana com Ajuda\n")
testeintegral(lambda start, end, func, N: quadgausseasy(start, end, func, N), 5)

Quadratura Gaussiana com Ajuda

Integral de x de 0 a 1
Valor obtido: 0.5.
Valor esperado: 0.5.
Diferença relativa: 0.00e+00%



Integral de x^2 de 0 a 1
Valor obtido: 0.33333333333333337.
Valor esperado: 0.3333333333333333.
Diferença relativa: 1.67e-14%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.400000000000001.
Valor esperado: 4.4.
Diferença relativa: 2.02e-14%



Integral de sin(x) de 0 a pi
Valor obtido: 2.0000001102844718.
Valor esperado: 2.
Diferença relativa: 5.51e-06%


In [219]:
print("Quadratura Gaussiana sem Ajuda 1\n")
testeintegral(lambda start, end, func, N: quadgauss1(start, end, func, N), 5)

Quadratura Gaussiana sem Ajuda 1

Integral de x de 0 a 1
Valor obtido: 0.5000000000000001.
Valor esperado: 0.5.
Diferença relativa: 2.22e-14%



Integral de x^2 de 0 a 1
Valor obtido: 0.3333333333333335.
Valor esperado: 0.3333333333333333.
Diferença relativa: 5.00e-14%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.4000000000000075.
Valor esperado: 4.4.
Diferença relativa: 1.61e-13%



Integral de sin(x) de 0 a pi
Valor obtido: 2.0000001117458153.
Valor esperado: 2.
Diferença relativa: 5.59e-06%


In [220]:
print("Quadratura Gaussiana sem Ajuda 2\n")
testeintegral(lambda start, end, func, N: quadgauss2(start, end, func, N), 5)

Quadratura Gaussiana sem Ajuda 2

Integral de x de 0 a 1
Valor obtido: 0.5002211734226559.
Valor esperado: 0.5.
Diferença relativa: 4.42e-02%



Integral de x^2 de 0 a 1
Valor obtido: 0.3335167008248806.
Valor esperado: 0.3333333333333333.
Diferença relativa: 5.50e-02%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.403839754119904.
Valor esperado: 4.4.
Diferença relativa: 8.73e-02%



Integral de sin(x) de 0 a pi
Valor obtido: 2.000393781374165.
Valor esperado: 2.
Diferença relativa: 1.97e-02%
